In [1]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../../HPCF/all/test_0.8/data/'
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorflow/

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/han/anaconda3/envs/fm/lib/python3.6/site

In [2]:
# Note that x is a 2D numpy array, 
# x[i, :] contains the user-item pair, and y[i] is the corresponding rating.

df = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
df_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))
train_y = np.loadtxt(os.path.join(dir_, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')
pop_count = pd.read_pickle(os.path.join(dir_, 'normalized_popularity_filter_track_5_user_100.pkl'))

num_users = len(df['uid'].unique())
num_tracks = len(df['tid'].unique())
print(num_users, num_tracks)

953 157567


In [3]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 1
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 953
Tag start position : 158520
Length of vector : 158521
----------------------------------------
Number of unique users : 953
Number of unique tracks : 157567


In [4]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

,tid,count,rating
70,70,17557,2980.75
1102,1102,6926,1484.25
83,83,6571,1796.75
13496,13496,6299,1669.75
952,952,6234,1527.50
2519,2519,6001,1772.25
4008,4008,5676,1008.25
5521,5521,5658,1727.25
3900,3900,5627,1475.50
2241,2241,5571,1183.75


In [5]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
70,70,17557,5.000000
1102,1102,6926,1.972433
83,83,6571,1.871333
13496,13496,6299,1.793871
952,952,6234,1.775360
2519,2519,6001,1.709005
4008,4008,5676,1.616449
5521,5521,5658,1.611323
3900,3900,5627,1.602495
2241,2241,5571,1.586547


In [6]:
len(pop_normalized['tid'].unique())

157567

In [7]:
pop = {}
for _, r in pop_normalized.iterrows():
    pop[r[0]] = r[2]

In [8]:
# pop[32528] = 7/max_rating*5
# pop[39438] = 20/max_rating*5
# pop[51291] = 5/max_rating*5

In [9]:
# pop
i = 0
v = []
sp_rows = []
sp_cols = []
for index, row in tqdm(df_train.iterrows(), total=len(df_train)):
    sp_rows.append(i)
    sp_cols.append(row['uid'])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row['tid'])
    v.append(1)

    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop[row['tid']])
        
    i += 1

train_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(len(df_train), l), dtype=float)

In [ ]:
# order = 2
# model = TFFMRegressor(
#     order=order, 
#     rank=6, 
#     optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
#     n_epochs=80, 
#     batch_size=16384,
#     init_std=0.00001,
#     reg=0.01,
#     input_type='sparse'
# )
# model.fit(train_x, train_y, show_progress=True)

In [10]:
order = 2
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.00001,
    reg=0.0001,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





100%|██████████| 10/10 [00:45<00:00,  4.56s/epoch]


In [11]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [12]:
x_test = []
for i in tqdm(range(unique_users)):
    user = df_train[df_train['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)

In [28]:
i = 0
v = []
sp_rows = []
sp_cols = []
for row in tqdm(x_test[:int(len(x_test)/4)]):
    sp_rows.append(i)
    sp_cols.append(row[0])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    v.append(1)
    
    sp_rows.append(i)
    sp_cols.append(tag_start)
    v.append(pop[row[1]])
    
    i += 1

In [29]:
test_x = sp.csr_matrix((v, (sp_rows, sp_cols)), shape=(int(len(v)/3), l), dtype=float)

In [30]:
v = []
sp_rows = []
sp_cols = []

In [31]:
pred1 = model.predict(test_x)

In [90]:
len(pred4), pred4

(36924411,
 array([1.5393249 , 1.2484418 , 0.85931677, ..., 0.41026658, 2.1864228 ,
        2.0089872 ], dtype=float32))

In [91]:
import pickle
open_file = open(os.path.join('../data/all (count)', 'pred4.pkl'), "wb")
pickle.dump(pred4, open_file)
open_file.close()

In [32]:
test_x = []

In [33]:
pred1 = list(pred1)
pred2 = list(pred2)
pred3 = list(pred3)
pred4 = list(pred4)

In [34]:
pred1.extend(pred2)
print(len(pred1))
pred1.extend(pred3)
print(len(pred1))
pred1.extend(pred4)
print(len(pred1))

73848820
110773230
147697641


In [35]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', pred1, False) 
df2[:10]

,uid,tid,rating
0,0,9,0.933119
1,0,15,0.619296
2,0,22,0.440176
3,0,28,0.743842
4,0,33,0.628341
5,0,38,0.389607
6,0,48,1.217015
7,0,53,0.350106
8,0,54,0.328016
9,0,56,0.382846


In [36]:
df2.to_pickle(os.path.join('../data/all (count)', 'topN_pred_' + file_name[:-3] + 'pkl'))

In [98]:
print(len(x_test))

147697641


In [48]:
dir_ = '../data/'
folder = 'sp_matrix_pop(old)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
7,0,3,0.25,1
11,0,7,2.50,3
32,0,18,1.50,2
34,0,20,2.50,3
45,0,31,5.00,17


In [49]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(unique_users)):
    prediction = df2[df2['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(unique_users)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/unique_users)


0.1884090909090909


In [50]:
df2.sort_values(by=['rating'],  ascending=False)

,uid,tid,rating
9543049,182,166,6.258834
2159719,41,166,6.223020
161196,3,166,6.214622
11001011,210,166,6.210902
3879732,74,166,6.201504
...,...,...,...
6195877,117,53496,0.196293
6193972,117,51585,0.195922
6191599,117,49211,0.195816
6179038,117,36636,0.195515
